<a href="https://colab.research.google.com/github/Saroj12Dangol/Document-Scanner/blob/Grammar-Correction/transformer_based_grammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 25.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.4 MB/s eta 0:00:00


In [ ]:
from happytransformer import HappyTextToText

happy_tt = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

Dataset jfleg downloaded and prepared to /root/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [ ]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [ ]:
import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])

In [ ]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [ ]:
before_result = happy_tt.eval("eval.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-95384b0ab69837da/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
print("Before loss:", before_result.loss)

Before loss: 1.2803919315338135


In [ ]:
from happytransformer import TTTrainArgs

args = TTTrainArgs(batch_size=8, num_train_epochs=12 )
train_results = happy_tt.train("train.csv", args=args)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

PyTorch: setting up devices
***** Running training *****
  Num examples = 3016
  Num Epochs = 12
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4524
  Number of trainable parameters = 222903552


Step,Training Loss
500,0.451400
1000,0.375100
1500,0.321200
2000,0.286000
2500,0.260000
3000,0.250500
3500,0.230300
4000,0.219700
4500,0.216500




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:


before_loss = happy_tt.eval("eval.csv")



print("After loss: ", before_loss.loss)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


After loss:  0.49175819754600525


In [4]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=2, max_length=100)

In [ ]:
input_text_1 = "grammar: I I wants to codes."
output_text_1 = happy_tt.generate_text(input_text_1, args=beam_settings)
print(output_text_1.text)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



I want to code.


In [ ]:
input_text_2 = "grammar: You're laptops is broken "
output_text_2 = happy_tt.generate_text(input_text_2, args=beam_settings)
print(output_text_2.text)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Your laptop is broken .


In [ ]:
input_text_3 = "grammar: You's  laptops is broke "
output_text_3 = happy_tt.generate_text(input_text_2, args=beam_settings)
print(output_text_3.text)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Your laptop is broken .


In [ ]:
from happytransformer import HappyGeneration
# ---------------------------------------------------------
# happy_gen = HappyGeneration(model_type="T5", model_name="happy_tt")
happy_tt.save("model/grammar")

Configuration saved in model/grammar/config.json
Configuration saved in model/grammar/generation_config.json
Model weights saved in model/grammar/pytorch_model.bin
tokenizer config file saved in model/grammar/tokenizer_config.json
Special tokens file saved in model/grammar/special_tokens_map.json
Copy vocab file to model/grammar/spiece.model


In [2]:
from happytransformer import HappyTextToText
grammar = HappyTextToText("T5", "/content/drive/MyDrive/model/grammar")

In [7]:
input_text_3 = "grammar: You's  laptops is broke "
output_text_3 = grammar.generate_text(input_text_3, args=beam_settings)
print(output_text_3.text)

Your laptop is broken .
